# Params for Simple_RNN

$params = cell\times input\_size + cell^2$

$cell^2$ means cells are fullly connected to themselves   meaning?


In [0]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow import keras
import numpy as np

In [2]:
cell = 1
input_size = 2
time_steps = 1    # doesn't change the number of params

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        #Dense(1, activation='relu', use_bias=False)    
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn (SimpleRNN)       (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [3]:
model.get_weights()

[array([[-1.0456531 ],
        [-0.19413507]], dtype=float32), array([[-1.]], dtype=float32)]

In [0]:
model.set_weights([[[2],[3]],[[5]]])

In [5]:
x = np.array([1, 0])
x = x.reshape(1,1,2)
model.predict(x)

array([[2.]], dtype=float32)

# Add a time step

In [6]:
cell = 1
input_size = 2
time_steps = 2    # doesn't change the number of params

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        #Dense(1, activation='relu', use_bias=False)    
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.set_weights([[[2],[3]],[[5]]])

In [8]:
x = np.array([1, 0, 0, 0])
x = x.reshape(1,2,2)
print(x)
model.predict(x)

[[[1 0]
  [0 0]]]


array([[10.]], dtype=float32)

In [9]:
x = np.array([1, 0, 1, 0])
x = x.reshape(1,2,2)
print(x)
model.predict(x)

[[[1 0]
  [1 0]]]


array([[12.]], dtype=float32)

In [10]:
x = np.array([1, 0, 0, 1])
x = x.reshape(1,2,2)
print(x)
model.predict(x)

[[[1 0]
  [0 1]]]


array([[13.]], dtype=float32)

# Add another timestep

In [11]:
cell = 1
input_size = 2
time_steps = 3    # doesn't change the number of params

model = keras.Sequential([
        SimpleRNN(cell, activation='relu', use_bias=False, input_shape=(time_steps, input_size)),
        #Dense(1, activation='relu', use_bias=False)    
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_2 (SimpleRNN)     (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.set_weights([[[2],[3]],[[5]]])

In [13]:
x = np.array([1, 0, 0, 1, 7, 11])
x = x.reshape(1,3,2)
print(x)
model.predict(x) == 2 * 7 + 3 * 11 + 5 * 13

[[[ 1  0]
  [ 0  1]
  [ 7 11]]]


array([[ True]])

# Add another input

In [14]:
x = np.array([1, 0, 0, 1, 7, 11, 1, 0, 0, 1, 7, 11])
x = x.reshape(2,3,2)
print(x)
model.predict(x) == 2 * 7 + 3 * 11 + 5 * 13

[[[ 1  0]
  [ 0  1]
  [ 7 11]]

 [[ 1  0]
  [ 0  1]
  [ 7 11]]]


array([[ True],
       [ True]])

# Prepare Training Data

In [15]:
def get_bit(n, bit):
    a = []
    for i in range(bit):
        a.append(n & 1)
        n >>= 1
    return a
  
for i in range(3):
    print(get_bit(i, 8))

[0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 0, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0]


In [0]:
time_steps = 8

x = []
y = []
for i in range(2**time_steps):
    for j in range(2**time_steps):
        i_b = get_bit(i, time_steps)
        j_b = get_bit(j, time_steps)
        x.append([[i_b[i], j_b[i]] for i in range(time_steps)])
        y.append([[k] for k in get_bit(i+j, 8)])
    

In [17]:
i = 256*11 + 5 # 256*3 + 5
print(x[i])
print(y[i])

[[1, 1], [1, 0], [0, 1], [1, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
[[0], [0], [0], [0], [1], [0], [0], [0]]


In [0]:
#for i in range(len(x)):
#  print(x[i], y[i])

In [19]:
import numpy as np

x = np.array(x)
x.shape

(65536, 8, 2)

In [20]:
y = np.array(y)
y.shape

(65536, 8, 1)

# Train

In [21]:
cell = 8
input_size = 2
time_steps = 8    # doesn't change the number of params

model = keras.Sequential([
        #SimpleRNN(cell, activation='relu', return_sequences=True, use_bias=False, input_shape=(time_steps, input_size)),
        LSTM(cell, activation='relu', return_sequences=True, use_bias=False, input_shape=(time_steps, input_size)),
        Dense(1, activation='relu', use_bias=False)    
])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 8, 8)              320       
_________________________________________________________________
dense (Dense)                (None, 8, 1)              8         
Total params: 328
Trainable params: 328
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer='adam', # tf.keras.optimizers.SGD(lr=0.01),
              loss='mean_squared_error',
              metrics=['accuracy'])

In [23]:
model.fit(x, y, epochs=10)  

Epoch 1/10
65536/65536 [==============================] - 11s 170us/step - loss: 0.2086 - acc: 0.6860
Epoch 2/10
65536/65536 [==============================] - 10s 156us/step - loss: 0.0049 - acc: 0.9980
Epoch 3/10
65536/65536 [==============================] - 10s 157us/step - loss: 3.6902e-04 - acc: 1.0000
Epoch 4/10
65536/65536 [==============================] - 10s 159us/step - loss: 1.4697e-04 - acc: 1.0000
Epoch 5/10
65536/65536 [==============================] - 10s 159us/step - loss: 8.1410e-05 - acc: 1.0000
Epoch 6/10
65536/65536 [==============================] - 10s 156us/step - loss: 5.3531e-05 - acc: 1.0000
Epoch 7/10
65536/65536 [==============================] - 10s 157us/step - loss: 3.9429e-05 - acc: 1.0000
Epoch 8/10
65536/65536 [==============================] - 10s 158us/step - loss: 3.0856e-05 - acc: 1.0000
Epoch 9/10
65536/65536 [==============================] - 10s 157us/step - loss: 2.4009e-05 - acc: 1.0000
Epoch 10/10
65536/65536 [=============================

In [24]:

count = 0


for i in range(len(x)):
  xi = x[i].reshape(1, 8, 2)
  yi = model.predict(xi).reshape(8, 1)
  for j in range(len(yi)):
    if y[i][j][0] != (0 if yi[j][0] < 0.5 else 1):
      count += 1
      break
count




0